<a href="https://colab.research.google.com/github/Cristinamulas/transfer_learning_keras/blob/main/model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import cv2

from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def plot_images_from_dir(path):
    """ it takes a path and return the plot fo each image"""
    images_classes = []
    for cls in os.listdir(path): # get all the classes in for each folder
         if cls != '.DS_Store':
            images_classes.append(cls)
    for i in images_classes: # loop over each class
        new_images = []
        my_folder = os.path.join(path,i)
        for file in os.listdir(os.path.join(path,i)):
            if file != '.DS_Store' and file != '.ipynb_checkpoints':
                new_images.append(cv2.imread(os.path.join(my_folder,file))) # convert the path into an array
        plt.figure(figsize=(50,50))
        columns = 10
        for i, image in enumerate(new_images):
            plt.subplot(len(new_images) / columns + 1, columns, i + 1)
            plt.imshow(image)
            plt.show()

In [ ]:
# folder = "/Users/cristinamulas/Desktop/transfer_learning_keras/data/val"

# plot_images_from_dir(folder)

In [ ]:
os.getcwd()
os.chdir('/content/drive/My Drive/transfer_learning_keras/data/')
os.getcwd()


'/content/drive/My Drive/transfer_learning_keras/data'

In [ ]:
base_path = os.getcwd() # get the working direntory
train_data_path = os.path.join(base_path + '/train') # add the new folder
train_data_path

test_data_path = os.path.join(base_path + '/test') # add the new folder
test_data_path

val_data_path = os.path.join(base_path + '/val') # add the new folder
val_data_path

'/content/drive/My Drive/transfer_learning_keras/data/val'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_generator = ImageDataGenerator(rescale=1./255,
                                     rotation_range=4,
                                   width_shift_range=0.1, 
                                   height_shift_range=0.1, 
                                   shear_range=0.3, 
                                   zoom_range=0.1, 
                                   horizontal_flip=True)
test_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
X_train_generator = train_generator.flow_from_directory(train_data_path, target_size=(32, 32), color_mode="rgb", batch_size = 32)
X_val_generator = test_generator.flow_from_directory(val_data_path, target_size=(32, 32), color_mode="rgb" )
X_test_generator = test_generator.flow_from_directory(test_data_path, target_size=(32, 32), color_mode="rgb",  batch_size =32)

Found 1356 images belonging to 3 classes.
Found 291 images belonging to 3 classes.
Found 39 images belonging to 3 classes.


In [ ]:
os.listdir('/Users/cristinamulas/Desktop/transfer_learning_keras/data/val')

['.DS_Store', 'metal', 'cardboard', '.ipynb_checkpoints', 'plastic']